In [7]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [9]:
tf.keras.backend.clear_session()

In [10]:
subject = 'Alagoas - Consumo de Cimento (t)'
start_index = 60
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [11]:
data = pd.read_csv('2003_mo_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Unnamed: 0,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - Consumo de Cimento (t)
0,2003-1,0.635160,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.294434,8.765721,30.779440,2.590528e+07,1.303625e+06,7.576443,2.359888e+07,30.109
1,2003-2,0.635571,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.296211,8.760196,30.599132,2.593685e+07,1.305020e+06,7.578654,2.360861e+07,27.684
2,2003-3,0.635982,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.297521,8.754671,30.545910,2.596843e+07,1.306414e+06,7.580866,2.361833e+07,26.534
3,2003-4,0.636393,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.298766,8.749145,30.377624,2.600000e+07,1.307809e+06,7.583078,2.362806e+07,25.109
4,2003-5,0.636804,0.648337,-5599.317941,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.299945,8.743620,30.464900,2.603157e+07,1.309203e+06,7.585289,2.363779e+07,25.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587645,NaN,24.556360,NaN,NaN,NaN,NaN,46.546
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587084,NaN,24.501314,NaN,NaN,NaN,NaN,47.781
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.585891,NaN,24.609100,NaN,NaN,NaN,NaN,51.171
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.584366,NaN,24.538488,NaN,NaN,NaN,NaN,49.126


In [12]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-2.146764,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-1.449375,-0.937577,-0.789529,-1.703997,-1.236973,-1.975005,-1.922747
1,-2.113412,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-1.423792,-0.939615,-0.803493,-1.687205,-1.181428,-1.949747,-1.897135
2,-2.080060,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-1.404924,-0.941652,-0.807615,-1.670414,-1.125883,-1.924488,-1.871523
3,-2.046707,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-1.386985,-0.943690,-0.820649,-1.653622,-1.070338,-1.899230,-1.845911
4,-2.013355,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-1.370013,-0.945728,-0.813889,-1.636831,-1.014793,-1.873971,-1.820299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.183322,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628,1.308074,0.919145,-1.198723,0.963258,-0.926262,0.929504,0.848917
200,1.177023,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775,1.359203,0.913671,-1.181757,0.946383,-0.898324,0.909729,0.830432
201,1.170724,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287,1.405650,0.908197,-1.171317,0.929508,-0.870386,0.889955,0.811947
202,1.164425,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156,1.459075,0.902724,-1.180748,0.912633,-0.842448,0.870180,0.793463


In [13]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      27.684
1      26.534
2      25.109
3      25.171
4      22.645
        ...  
235    47.781
236    51.171
237    49.126
238    49.126
239       NaN
Name: Alagoas - Consumo de Cimento (t), Length: 240, dtype: float64

In [14]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
60,-0.508441,-0.537241,-0.615167,0.042292,0.488894,-0.717137,-0.601292,-0.315948,0.240813,-0.733600,-1.019703,-0.037973,-0.848808,1.405264,-0.742184,-0.808534
61,-0.489848,-0.479316,-0.525870,0.011943,-0.072763,-0.714911,-0.580436,-0.286095,0.246566,-0.701180,-1.018834,-0.062281,-0.840179,1.413569,-0.737145,-0.804966
62,-0.471254,-0.466312,-0.510476,-0.019339,-0.316940,-0.694168,-0.559580,-0.256243,0.315327,-0.668084,-1.017965,-0.050358,-0.831550,1.421875,-0.732105,-0.801398
63,-0.452661,-0.438778,-0.537853,-0.059264,0.180777,-0.700209,-0.538723,-0.226390,0.356189,-0.646235,-1.017096,-0.050171,-0.822921,1.430181,-0.727066,-0.797829
64,-0.434068,-0.457839,-0.446299,-0.104671,-0.212484,-0.660717,-0.517867,-0.196537,0.524165,-0.619607,-1.016226,-0.021423,-0.814292,1.438487,-0.722026,-0.794261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.324514,-0.742853,0.615006,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806,1.121339,1.186919,-1.069189,1.177341,-1.214407,1.097518,1.109211
164,1.323316,-0.787367,0.546355,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523,1.103076,1.173733,-1.161695,1.181165,-1.230415,1.105489,1.111823
165,1.322118,-0.757940,0.421319,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175,1.083984,1.160546,-1.230868,1.184990,-1.246424,1.113460,1.114436
166,1.320921,-0.917469,0.661467,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146,1.070055,1.147360,-1.298996,1.188814,-1.262433,1.121431,1.117048


In [15]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

60     32.977000
61     35.662000
62     39.980000
63     34.089000
64     32.715000
         ...    
163    42.491371
164    41.535029
165    40.578686
166    39.622343
167    38.666000
Name: Alagoas - Consumo de Cimento (t), Length: 108, dtype: float64

In [16]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [17]:
reshaped_train, reshaped_target = create_batches(train_input, train_target, 36, 60)
reshaped_train.shape

(72, 36, 16)

In [18]:
# input para treinamento
test_input = input_data.iloc[train_split-36:split_index + 1]
test_input

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
132,0.384263,1.590448,0.225519,-0.298056,-0.311347,-0.077688,0.623805,0.982447,0.041401,1.053656,1.055214,1.307529,0.851394,-0.396992,0.813623,0.917513
133,0.384465,1.702213,0.413533,-0.281023,-0.313831,-0.030826,0.635788,0.979099,-0.019620,1.094151,1.070281,1.261277,0.867621,-0.443166,0.829740,0.930914
134,0.384667,1.677438,0.431751,-0.265304,0.006739,-0.076111,0.647771,0.975750,0.029284,1.134973,1.085348,1.213376,0.883848,-0.489340,0.845857,0.944315
135,0.384868,1.638599,0.414613,-0.251648,-0.486922,-0.205002,0.659754,0.972402,0.072516,1.179185,1.100415,1.163870,0.900074,-0.535515,0.861974,0.957716
136,0.385070,1.691843,0.566348,-0.231997,-0.559925,-0.132229,0.671737,0.969054,-0.064532,1.214733,1.115482,1.113252,0.916301,-0.581689,0.878091,0.971117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.183322,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628,1.308074,0.919145,-1.198723,0.963258,-0.926262,0.929504,0.848917
200,1.177023,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775,1.359203,0.913671,-1.181757,0.946383,-0.898324,0.909729,0.830432
201,1.170724,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287,1.405650,0.908197,-1.171317,0.929508,-0.870386,0.889955,0.811947
202,1.164425,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156,1.459075,0.902724,-1.180748,0.912633,-0.842448,0.870180,0.793463


In [19]:
reshaped_test, reshaped_test_target = create_batches(test_input, target_data, 36, 132)
reshaped_test.shape 

(36, 36, 16)

In [20]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * 6 + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [21]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 10)
    target,target_val = validation_splitter(train_target, 10)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36, activation='tanh'),
#         tf.keras.layers.Dropout(0.3),
#         tf.keras.layers.Dense(74, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [25]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [26]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[826092791, 411586994, 64430712, 3720459345, 509897118, 2358479996, 1347401479, 307239858, 1062134621, 4180249649, 1856155159, 22708025, 1533336304, 469167962, 4120287929, 1966344021, 1871307769, 2263922504, 618673017, 3980043828, 1115040065, 2691478379, 787242870, 875262187, 2120713604]


Step: 0 ___________________________________________
val_loss: 5.585084438323975
winner_seed: 826092791


Step: 1 ___________________________________________
val_loss: 7.955827236175537


Step: 2 ___________________________________________
val_loss: 6.670435428619385


Step: 3 ___________________________________________
val_loss: 6.069944858551025


Step: 4 ___________________________________________
val_loss: 3.57940936088562
winner_seed: 509897118


Step: 5 ___________________________________________
val_loss: 6.13593053817749


Step: 6 ___________________________________________
val_loss: 5.97615909576416


Step: 7 ___________________________________________
val_loss: 4.851016044616699


Step: 8 __

2023-10-10 18:29:38.160775: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 3.726447582244873


Step: 11 ___________________________________________
val_loss: 6.28671407699585


Step: 12 ___________________________________________
val_loss: 6.531646728515625


Step: 13 ___________________________________________
val_loss: 3.823045015335083


Step: 14 ___________________________________________
val_loss: 5.9521565437316895


Step: 15 ___________________________________________
val_loss: 5.744359016418457


Step: 16 ___________________________________________
val_loss: 4.325680255889893


Step: 17 ___________________________________________
val_loss: 7.048698425292969


Step: 18 ___________________________________________
val_loss: 3.3954174518585205
winner_seed: 618673017


Step: 19 ___________________________________________
val_loss: 1.794474720954895
winner_seed: 3980043828


Step: 20 ___________________________________________
val_loss: 5.3778076171875


Step: 21 ___________________________________________
val_loss: 4.880979061126709


Step: 22 __

In [27]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
3/3 [==============================] - 2s 164ms/step - loss: 3654.0491 - val_loss: 1330.0970
Epoch 2/10000
3/3 [==============================] - 0s 10ms/step - loss: 1229.0302 - val_loss: 222.5371
Epoch 3/10000
3/3 [==============================] - 0s 10ms/step - loss: 222.9594 - val_loss: 163.6093
Epoch 4/10000
3/3 [==============================] - 0s 10ms/step - loss: 105.5132 - val_loss: 168.6840
Epoch 5/10000
3/3 [==============================] - 0s 10ms/step - loss: 105.4378 - val_loss: 233.1879
Epoch 6/10000
3/3 [==============================] - 0s 10ms/step - loss: 114.1903 - val_loss: 162.7027
Epoch 7/10000
3/3 [==============================] - 0s 10ms/step - loss: 105.1429 - val_loss: 194.6650
Epoch 8/10000
3/3 [==============================] - 0s 10ms/step - loss: 100.7621 - val_loss: 240.8791
Epoch 9/10000
3/3 [==============================] - 0s 10ms/step - loss: 115.5672 - val_loss: 189.4308
Epoch 10/10000
3/3 [==============================] - 0s 10m

3/3 [==============================] - 0s 10ms/step - loss: 37.6620 - val_loss: 19.1595
Epoch 81/10000
3/3 [==============================] - 0s 10ms/step - loss: 38.1697 - val_loss: 14.1115
Epoch 82/10000
3/3 [==============================] - 0s 10ms/step - loss: 34.3928 - val_loss: 14.4060
Epoch 83/10000
3/3 [==============================] - 0s 10ms/step - loss: 36.0707 - val_loss: 36.6274
Epoch 84/10000
3/3 [==============================] - 0s 10ms/step - loss: 38.7494 - val_loss: 36.2747
Epoch 85/10000
3/3 [==============================] - 0s 10ms/step - loss: 40.0304 - val_loss: 32.9887
Epoch 86/10000
3/3 [==============================] - 0s 9ms/step - loss: 40.0650 - val_loss: 18.2631
Epoch 87/10000
3/3 [==============================] - 0s 10ms/step - loss: 33.5846 - val_loss: 20.5021
Epoch 88/10000
3/3 [==============================] - 0s 10ms/step - loss: 41.5093 - val_loss: 27.2031
Epoch 89/10000
3/3 [==============================] - 0s 9ms/step - loss: 38.1278 - val_l

3/3 [==============================] - 0s 11ms/step - loss: 41.8704 - val_loss: 28.7995
Epoch 160/10000
3/3 [==============================] - 0s 11ms/step - loss: 40.1841 - val_loss: 27.7045
Epoch 161/10000
3/3 [==============================] - 0s 11ms/step - loss: 35.4531 - val_loss: 21.2141
Epoch 162/10000
3/3 [==============================] - 0s 11ms/step - loss: 36.8607 - val_loss: 26.5419
Epoch 163/10000
3/3 [==============================] - 0s 11ms/step - loss: 35.5360 - val_loss: 14.5282
Epoch 164/10000
3/3 [==============================] - 0s 10ms/step - loss: 37.2469 - val_loss: 26.5301
Epoch 165/10000
3/3 [==============================] - 0s 11ms/step - loss: 37.1564 - val_loss: 16.4863
Epoch 166/10000
3/3 [==============================] - 0s 10ms/step - loss: 35.5467 - val_loss: 14.4480
Epoch 167/10000
3/3 [==============================] - 0s 11ms/step - loss: 38.3032 - val_loss: 13.8710
Epoch 168/10000
3/3 [==============================] - 0s 10ms/step - loss: 32.5

Epoch 238/10000
3/3 [==============================] - 0s 11ms/step - loss: 35.8338 - val_loss: 22.1195
Epoch 239/10000
3/3 [==============================] - 0s 11ms/step - loss: 31.4367 - val_loss: 23.3360
Epoch 240/10000
3/3 [==============================] - 0s 11ms/step - loss: 32.5685 - val_loss: 34.8088
Epoch 241/10000
3/3 [==============================] - 0s 11ms/step - loss: 38.5517 - val_loss: 32.9090
Epoch 242/10000
3/3 [==============================] - 0s 12ms/step - loss: 39.1681 - val_loss: 31.2614
Epoch 243/10000
3/3 [==============================] - 0s 11ms/step - loss: 37.4086 - val_loss: 27.0565
Epoch 244/10000
3/3 [==============================] - 0s 11ms/step - loss: 36.3264 - val_loss: 26.2110
Epoch 245/10000
3/3 [==============================] - 0s 11ms/step - loss: 34.3231 - val_loss: 20.9717
Epoch 246/10000
3/3 [==============================] - 0s 10ms/step - loss: 34.8097 - val_loss: 22.2976
Epoch 247/10000
3/3 [==============================] - 0s 11ms/s

Epoch 317/10000
3/3 [==============================] - 0s 12ms/step - loss: 39.5657 - val_loss: 21.5385
Epoch 318/10000
3/3 [==============================] - 0s 10ms/step - loss: 34.3106 - val_loss: 18.5642
Epoch 319/10000
3/3 [==============================] - 0s 11ms/step - loss: 31.9770 - val_loss: 15.8775
Epoch 320/10000
3/3 [==============================] - 0s 11ms/step - loss: 33.6052 - val_loss: 14.8311
Epoch 321/10000
3/3 [==============================] - 0s 11ms/step - loss: 32.4556 - val_loss: 18.3313
Epoch 322/10000
3/3 [==============================] - 0s 10ms/step - loss: 31.9293 - val_loss: 18.5398
Epoch 323/10000
3/3 [==============================] - 0s 10ms/step - loss: 29.9251 - val_loss: 16.0401
Epoch 324/10000
3/3 [==============================] - 0s 11ms/step - loss: 31.6015 - val_loss: 47.2097
Epoch 325/10000
3/3 [==============================] - 0s 10ms/step - loss: 52.4131 - val_loss: 22.6843
Epoch 326/10000
3/3 [==============================] - 0s 11ms/s

...

3/3 [==============================] - 0s 11ms/step - loss: 27.8049 - val_loss: 11.1743
Epoch 2120/10000
3/3 [==============================] - 0s 11ms/step - loss: 26.0006 - val_loss: 9.8557
Epoch 2121/10000
3/3 [==============================] - 0s 11ms/step - loss: 26.8339 - val_loss: 17.1300
Epoch 2122/10000
3/3 [==============================] - 0s 10ms/step - loss: 30.3857 - val_loss: 12.9981
Epoch 2123/10000
3/3 [==============================] - 0s 11ms/step - loss: 26.7313 - val_loss: 16.4750
Epoch 2124/10000
3/3 [==============================] - 0s 11ms/step - loss: 28.2001 - val_loss: 12.4228
Epoch 2125/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.4957 - val_loss: 12.6027
Epoch 2126/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.3198 - val_loss: 12.4418
Epoch 2127/10000
3/3 [==============================] - 0s 10ms/step - loss: 26.0205 - val_loss: 12.5071
Epoch 2128/10000
3/3 [==============================] - 0s 11ms/step - lo

3/3 [==============================] - 0s 11ms/step - loss: 28.1785 - val_loss: 14.0480
Epoch 2198/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.5079 - val_loss: 10.6449
Epoch 2199/10000
3/3 [==============================] - 0s 11ms/step - loss: 27.3051 - val_loss: 18.9892
Epoch 2200/10000
3/3 [==============================] - 0s 11ms/step - loss: 51.5748 - val_loss: 5.2674
Epoch 2201/10000
3/3 [==============================] - 0s 12ms/step - loss: 32.5541 - val_loss: 7.5017
Epoch 2202/10000
3/3 [==============================] - 0s 11ms/step - loss: 29.7985 - val_loss: 20.2800
Epoch 2203/10000
3/3 [==============================] - 0s 11ms/step - loss: 27.3482 - val_loss: 15.5468
Epoch 2204/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.3116 - val_loss: 15.4764
Epoch 2205/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.5213 - val_loss: 13.4228
Epoch 2206/10000
3/3 [==============================] - 0s 11ms/step - los

3/3 [==============================] - 0s 11ms/step - loss: 24.8060 - val_loss: 14.5039
Epoch 2276/10000
3/3 [==============================] - 0s 11ms/step - loss: 24.4094 - val_loss: 13.8659
Epoch 2277/10000
3/3 [==============================] - 0s 11ms/step - loss: 24.7542 - val_loss: 15.3306
Epoch 2278/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.6674 - val_loss: 15.9902
Epoch 2279/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.7672 - val_loss: 14.0894
Epoch 2280/10000
3/3 [==============================] - 0s 10ms/step - loss: 25.2154 - val_loss: 13.2348
Epoch 2281/10000
3/3 [==============================] - 0s 11ms/step - loss: 28.8879 - val_loss: 12.0693
Epoch 2282/10000
3/3 [==============================] - 0s 10ms/step - loss: 27.9998 - val_loss: 13.9314
Epoch 2283/10000
3/3 [==============================] - 0s 10ms/step - loss: 27.1173 - val_loss: 21.6235
Epoch 2284/10000
3/3 [==============================] - 0s 11ms/step - l

3/3 [==============================] - 0s 11ms/step - loss: 25.4811 - val_loss: 16.1083
Epoch 2354/10000
3/3 [==============================] - 0s 11ms/step - loss: 28.5944 - val_loss: 13.9716
Epoch 2355/10000
3/3 [==============================] - 0s 11ms/step - loss: 26.3998 - val_loss: 11.1496
Epoch 2356/10000
3/3 [==============================] - 0s 12ms/step - loss: 25.6826 - val_loss: 14.7398
Epoch 2357/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.7269 - val_loss: 13.5644
Epoch 2358/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.8267 - val_loss: 13.1626
Epoch 2359/10000
3/3 [==============================] - 0s 11ms/step - loss: 24.7241 - val_loss: 13.2702
Epoch 2360/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.9687 - val_loss: 11.0125
Epoch 2361/10000
3/3 [==============================] - 0s 11ms/step - loss: 26.1880 - val_loss: 23.7246
Epoch 2362/10000
3/3 [==============================] - 0s 11ms/step - l

3/3 [==============================] - 0s 12ms/step - loss: 25.2792 - val_loss: 15.3406
Epoch 2432/10000
3/3 [==============================] - 0s 11ms/step - loss: 27.3633 - val_loss: 19.8049
Epoch 2433/10000
3/3 [==============================] - 0s 11ms/step - loss: 27.1103 - val_loss: 32.0773
Epoch 2434/10000
3/3 [==============================] - 0s 12ms/step - loss: 35.8645 - val_loss: 19.6057
Epoch 2435/10000
3/3 [==============================] - 0s 11ms/step - loss: 27.7682 - val_loss: 20.2947
Epoch 2436/10000
3/3 [==============================] - 0s 12ms/step - loss: 27.1192 - val_loss: 18.5161
Epoch 2437/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.9001 - val_loss: 17.4127
Epoch 2438/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.4747 - val_loss: 18.1635
Epoch 2439/10000
3/3 [==============================] - 0s 11ms/step - loss: 25.3363 - val_loss: 19.8799
Epoch 2440/10000
3/3 [==============================] - 0s 11ms/step - l

In [28]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [29]:
errors, mae, mape = mae_mape_calculator(trained_model, 
                                        reshaped_test, 
                                        reshaped_test_target, 0)

1/1 [==============================] - 0s 316ms/step
Month-1 - Error: [[6.554268]]
1/1 [==============================] - 0s 15ms/step
Month-2 - Error: [[3.1508675]]
1/1 [==============================] - 0s 14ms/step
Month-3 - Error: [[6.693102]]
1/1 [==============================] - 0s 15ms/step
Month-4 - Error: [[9.95874]]
1/1 [==============================] - 0s 15ms/step
Month-5 - Error: [[15.527355]]
1/1 [==============================] - 0s 15ms/step
Month-6 - Error: [[16.358486]]
1/1 [==============================] - 0s 14ms/step
Month-7 - Error: [[10.152554]]
1/1 [==============================] - 0s 14ms/step
Month-8 - Error: [[7.67]]
1/1 [==============================] - 0s 15ms/step
Month-9 - Error: [[4.381586]]
1/1 [==============================] - 0s 15ms/step
Month-10 - Error: [[3.2695713]]
1/1 [==============================] - 0s 14ms/step
Month-11 - Error: [[2.214552]]
1/1 [==============================] - 0s 14ms/step
Month-12 - Error: [[4.2011147]]
1/1 [======

In [30]:
display(mae)
display(mape)

23.973074

0.7769176

In [33]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum} ")
            target_sum = 0
            prediction_sum = 0
            
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum} ")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [34]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 0)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[425.1151]] - Target[354.11599999999993]| =  Error: [[70.999115]]; MAPE:[[0.20049678]] 
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[18.715946]] - Target[342.67499999999995]| =  Error: [[323.95905]]; MAPE:[[0.94538283]] 
1/1 [==============================] - 0s 14ms/step
Ano-11: |Prediction[[-42.64545]] - Target[406.294]| =  Error: [[448.93945]]; MAPE:[[1.104962]] 


281.2992

0.75028056